In [2]:
from openai import OpenAI
import os
from dotenv import dotenv_values
from ReadLoad import read_jsonl, write_jsonl, read_json,write_json
from tqdm import tqdm
from evaluation import get_embedding, cosine_similarity
from prompt_template import get_sys_prompt, get_qa_prompt
from shusheng import  get_respond
import pandas as pd
import time

config = dotenv_values('.env')
client = OpenAI(
    api_key=config['qwen_key'],
    base_url=config['qwen_url']
)


def get_response(prompt):
    time.sleep(0.1)
    try:
        completion = client.chat.completions.create(
            model="qwen2-7b-instruct",
            messages=[
                {'role': 'system', 'content': get_sys_prompt()},
                {'role': 'user', 'content': prompt}
            ]

            )
        return completion.choices[0].message.content
    except Exception as e:
    # Handle any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
        return f"An unexpected error occurred: {e}"


Downloading: 100%|██████████| 916/916 [00:00<00:00, 2.04kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 65.2B/s]
Downloading: 100%|██████████| 8.64k/8.64k [00:00<00:00, 11.6kB/s]
Downloading: 100%|██████████| 123/123 [00:00<00:00, 199B/s]
Downloading: 100%|██████████| 1.82G/1.82G [00:07<00:00, 257MB/s] 
Downloading: 100%|██████████| 1.81G/1.81G [00:08<00:00, 241MB/s] 
Downloading: 100%|██████████| 1.84G/1.84G [00:07<00:00, 273MB/s] 
Downloading: 100%|██████████| 1.81G/1.81G [00:07<00:00, 254MB/s] 
Downloading: 100%|██████████| 1.84G/1.84G [00:08<00:00, 224MB/s] 
Downloading: 100%|██████████| 1.81G/1.81G [00:15<00:00, 124MB/s] 
Downloading: 100%|██████████| 1.84G/1.84G [00:08<00:00, 239MB/s] 
Downloading: 100%|██████████| 1.63G/1.63G [00:06<00:00, 276MB/s] 
Downloading: 100%|██████████| 17.8k/17.8k [00:00<00:00, 33.9kB/s]
Downloading: 100%|██████████| 78.8k/78.8k [00:00<00:00, 156kB/s]
Downloading: 100%|██████████| 17.1k/17.1k [00:00<00:00, 33.8kB/s]
Downloading: 100%|██████

AssertionError: Torch not compiled with CUDA enabled

In [11]:
def completion(data):
    for d in tqdm(data):
        query = d['问题']
        clause = d['条款']
        name = d['产品名']
        prompt = get_sys_prompt() + '\n' +get_qa_prompt(name,clause, query)
        d['prompt'] = prompt
        #d['answer'] = get_response(prompt)
        d['answer'] = get_respond(prompt)
    return data

In [12]:
def evaluation(data, name="测试数据"):
    similaritys = []
    for d in tqdm(data):
        vec1 = dev_data_embd[d['ID']]
        vec2 = get_embedding(d['answer'])
        similarity = cosine_similarity(vec1, vec2)
        d['similarity'] = similarity
        similaritys.append(similarity)
    length = len(data)
    score = sum(similaritys)/length
    print(f'测评数据集：{length} 模型推理结果与答案的相识度得分：{score}')
    df = pd.DataFrame(data)
    df.to_excel(f"{name}_score_{score}.xlsx", index=False)
    return data

In [13]:
def to_summit_json(data, commit="大道至简_result"):
    result = []
    for d in data:
        rd = {
            "ID": d['ID'],
            "question": d['问题'],
            "answer": d['answer']
        }
        result.append(rd)
    write_json(result, commit)

### loading data

In [14]:
#test_data = read_json("dataset/test.json")
dev_data = read_jsonl("dataset/resultdev_with_embedding.jsonl")
dev_data_embd = {}
for d in dev_data:
    dev_data_embd[d['ID']] = d['ans_embedding']
    d.pop('ans_embedding')

In [ ]:
comp_data = completion(dev_data[200:300])
#comp_data
eval_data = evaluation(comp_data)
# to_summit_json(eval_data)

 42%|████▏     | 42/100 [05:56<05:42,  5.90s/it]

In [5]:
get_respond("你是谁")

'我是书生·浦语，一个由上海人工智能实验室开发的语言模型。我的设计理念是帮助用户，诚实，无害。我能够理解并流利地使用用户选择的语言，例如英语和中文。'

### 参考资料

https://dashscope.console.aliyun.com/billing